<a href="https://colab.research.google.com/github/plaban1981/Hugging_Face_transformers_topics/blob/main/Predictive_Writing_using_GPT_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initial Fine-tuning with GPT Neo 125M parameter model

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 8.1 MB/s 
     |████████████████████████████████| 6.6 MB 45.2 MB/s 
     |████████████████████████████████| 120 kB 71.0 MB/s 


In [12]:
text = '''Fashion is an essential aspect of the modern society'''

## Import Required libraries

In [3]:
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForCausalLM, IntervalStrategy


## Hugging face provides pre-trained GPT-Neo model and its corresponding tokenizer.

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M")

In [28]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
text = "I enjoy experimenting with"

In [23]:
text = '''Fashion is an essential aspect of the modern society. It is inherently present in all forms of the media, it influences personal choices, and creates mainstream tendencies. Although it is believed that designers are the ones who create fashion, thus enforcing their view onto the general public, popular shifts and trends of social life have a great impact on how clothing designs are developed. Therefore, fashion is important because it affects people while people shape it in return, which creates a never-ending cycle of co-dependency.

Fashion designers dictate which type of clothing is acceptable and which is not. Their choices are not based on rules, conventions, or even common sense, but rather on their personal vision and inspiration. As this vision is promoted through the media, people come to believe that they have to wear particular clothes and accessories in order to lead successful social lives. In this way, a few creators decide what the majority of people in communities and whole countries are going to wear in the near future. It does not matter whether new fashion trends correspond to individual tastes and preferences, so it is often the case when one willingly wears something uncomfortable or unsuitable only because popular clothing designers and brands invented it. To wear something else is to stand out and attract negative attention, and not all people are ready for these unfavorable consequences. As a result, societies are shaped by local and global fashion trends, for they gradually accept the enforced perceptions of attractiveness and beauty. These perceptions inevitably influence other aspects of social life, such as communication, stereotypical thinking, and social labels.

On the other hand, fashion designers have to draw their inspiration from already popular social trends in order to succeed. For example, it is often the case when they use concepts from pop culture and implement them in clothing design. This is done to make a new trend appealing, for if the general public connects it to something which has already gained recognition, the number of sales is bound to increase. The application of this basic marketing principle allows people to influence fashion instead of being nothing more than its mindless followers. If the resulting clothing trend is uncomfortable or even scandalous, fashion designers are not the only ones to blame. Whether people realize it or not, their current everyday choices may predict what they are going to wear next season.

All in all, it is impossible to deny the importance of fashion when it evidently creates the connection between trends and people. On the one hand, it may dictate us what we should wear, initiate prejudice, and reconstruct popular perception of beauty, thus demonstrating the power of its impact. On the other hand, common social choices and tendencies inevitably influence fashion trends because designers aim to use already popular ideas in their original clothing lines. Therefore, it may be said that fashion is more than clothes and accessories because it is one of the most fundamental elements of human society. '''

In [24]:
input_ids = tokenizer(text,return_tensors='pt')

In [25]:
input_ids

{'input_ids': tensor([[   37,  5880,   318,   281,  6393,  4843,   286,   262,  3660,  3592,
            13,   632,   318, 20108,  1944,   287,   477,  5107,   286,   262,
          2056,    11,   340, 16717,  2614,  7747,    11,   290,  8075,  8661,
         25671,    13,  4900,   340,   318,  4762,   326, 15441,   389,   262,
          3392,   508,  2251,  6977,    11,  4145, 26587,   511,  1570,  4291,
           262,  2276,  1171,    11,  2968, 15381,   290, 11257,   286,  1919,
          1204,   423,   257,  1049,  2928,   319,   703,  9528,  9824,   389,
          4166,    13,  8447,    11,  6977,   318,  1593,   780,   340, 10975,
           661,   981,   661,  5485,   340,   287,  1441,    11,   543,  8075,
           257,  1239,    12,  1571,  6772,   286,   763,    12, 45841,  1387,
            13,   198,   198,    37,  5880, 15441, 27861,   543,  2099,   286,
          9528,   318, 10909,   290,   543,   318,   407,    13,  5334,  7747,
           389,   407,  1912,   319,  

In [26]:
input_ids['input_ids']

tensor([[   37,  5880,   318,   281,  6393,  4843,   286,   262,  3660,  3592,
            13,   632,   318, 20108,  1944,   287,   477,  5107,   286,   262,
          2056,    11,   340, 16717,  2614,  7747,    11,   290,  8075,  8661,
         25671,    13,  4900,   340,   318,  4762,   326, 15441,   389,   262,
          3392,   508,  2251,  6977,    11,  4145, 26587,   511,  1570,  4291,
           262,  2276,  1171,    11,  2968, 15381,   290, 11257,   286,  1919,
          1204,   423,   257,  1049,  2928,   319,   703,  9528,  9824,   389,
          4166,    13,  8447,    11,  6977,   318,  1593,   780,   340, 10975,
           661,   981,   661,  5485,   340,   287,  1441,    11,   543,  8075,
           257,  1239,    12,  1571,  6772,   286,   763,    12, 45841,  1387,
            13,   198,   198,    37,  5880, 15441, 27861,   543,  2099,   286,
          9528,   318, 10909,   290,   543,   318,   407,    13,  5334,  7747,
           389,   407,  1912,   319,  3173,    11, 2

## Raw model Inference
* how the model behaves w/o any fine-tuning:

In [27]:
outputs=model.generate(input_ids['input_ids'],
                      max_length=133,
                      do_sample=True,
                      top_p=0.95,
                      # num_beams=5,
                      temperature=0.7,
                      no_repeat_ngram_size=3,
                      num_return_sequences=5
                      )
for i,out in enumerate(outputs):
  print(f"Sugestion{i} :{tokenizer.decode(out, skip_special_tokens=True)}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 570, but `max_length` is set to 133. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


Sugestion0 :Fashion is an essential aspect of the modern society. It is inherently present in all forms of the media, it influences personal choices, and creates mainstream tendencies. Although it is believed that designers are the ones who create fashion, thus enforcing their view onto the general public, popular shifts and trends of social life have a great impact on how clothing designs are developed. Therefore, fashion is important because it affects people while people shape it in return, which creates a never-ending cycle of co-dependency.

Fashion designers dictate which type of clothing is acceptable and which is not. Their choices are not based on rules, conventions, or even common sense, but rather on their personal vision and inspiration. As this vision is promoted through the media, people come to believe that they have to wear particular clothes and accessories in order to lead successful social lives. In this way, a few creators decide what the majority of people in commu

* https://medium.com/@sharmavarun.cs/predictive-writing-using-gpt-transformer-a042d37f7fb3

## Data Preprocessing

In [ ]:
# Read the data
df=pd.read_excel("/home/ec2-user/SageMaker/GPT/Dataset/admission_essays_org1.xlsx")

# Cleaning the essay data
#Since the dataset contains certain special asterisk characters ‘*’, 
#they need to be replaced by a placeholder of ‘<name>’.
import re
re.sub(r"[*]+","<name>","i went to *** dfsdf")

df["essay_text"]=df["essay_text"].apply(lambda x:re.sub(r"[*]+","<name>",x))

## Fine-tuning on Essay Dataset

In [ ]:
class custom_Dataset():
    def __init__(self, dataframe, tokenizer, source_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.input_text = dataframe["essay_text"].values

    def __len__(self):
        return len(self.input_text)

    def __getitem__(self,index):
        input_text = str(self.input_text[index])

        # cleaning data so as to ensure data is in string type
        input_text = " ".join(input_text.split())

        source = self.tokenizer.encode_plus(
            input_text,
            max_length=self.source_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        source_ids = source["input_ids"].squeeze()
        source_mask = source["attention_mask"].squeeze()

        return (source_ids.to(dtype=torch.long), source_mask.to(dtype=torch.long),)

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

dataset = custom_Dataset(df, tokenizer, source_len=512)
train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

## Training Parameters

In [ ]:
learning_rate = 1e-5
warmup_steps = 1e2
epsilon = 1e-8
epochs =5

## optimizer

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = learning_rate,
                  eps = epsilon
                )

##Train the model

In [ ]:
for epoch_i in range(epochs):
    print("")
    print('Training...')

    total_train_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):
        model.zero_grad()
        input_ids = batch[0].to(device)
        attn_ids=batch[1].to(device)
        model.zero_grad()   

        outputs = model(input_ids,
                        labels=input_ids,
                        attention_mask = attn_ids
                        )
        # print(outputs)
        loss = outputs[0]  
        batch_loss = loss
        total_train_loss += batch_loss.item()
        loss.backward()
        optimizer.step()
        del batch
        if step%200==0:
            print(f"Step:{step}=======Losss:{loss}")
    print('======== Epoch {:} / {:} ======== LOSS... {}======Perplexity: {}'.format(epoch_i + 1, epochs,total_train_loss, math.exp(total_train_loss/step)))


##  Inference from the Fine-tuned model


In [ ]:
torch.save(model,"gpt_neo_small_US.pth")

input_sentence="I am a student in US"

seq=tokenizer(input_sentence,return_tensors='pt')['input_ids'].to(device)
outputs=model.generate(seq,
                      max_length=20,
                      do_sample=True,
                      top_p=0.95,
                      temperature=0.7,
                      no_repeat_ngram_size=3,
                      num_return_sequences=5
                      ).to(device)
for i,out in enumerate(outputs):
  print(f"Sugestion{i} :{tokenizer.decode(out, skip_special_tokens=True)}")